In [1]:
import arcpy
import psycopg2
from psycopg2 import sql
from psycopg2.extras import RealDictCursor

In [ ]:
conn= psycopg2.connect(host = '35.223.186.20',
                             database = 'lab0',
                             user = 'postgres',
                             password = '')
cur = conn.cursor()

create_table_query = sql.SQL("CREATE TABLE table2 (polygon geometry)")
cur.execute(create_table_query)

conn.commit()
cur.close()
conn.close()

In [ ]:
# Create a polygon using ArcPy geometry primitives
polygon = arcpy.Polygon(arcpy.Array([arcpy.Point(1, 1), arcpy.Point(1, 2), arcpy.Point(2, 2), arcpy.Point(2, 1)]))

# Convert the polygon to WKT
wkt = polygon.WKT

# Connect to the PostGIS database using psycopg2
conn= psycopg2.connect(host = '35.223.186.20',
                             database = 'lab0',
                             user = 'postgres',
                             password = '')

# Create a cursor
cur = conn.cursor()

# Define the insert statement
insert_stmt = "INSERT INTO mytable (geom) VALUES (ST_GeomFromText('{}', 4326))".format(wkt)


# Execute the insert statement
cur.execute(insert_stmt)

# Commit the transaction
conn.commit()

# Close the cursor and the database connection
cur.close()
conn.close()


In [ ]:
sudo apt-get update
sudo apt-get install postgis
pip install psycopg2-binary
pip install Flask


In [ ]:
#make a .py file of the following file and upload to VM ssh

import psycopg2
from flask import Flask, jsonify
from psycopg2 import sql
from psycopg2.extras import RealDictCursor

app = Flask(__name__)

@app.route('/get_polygon', methods=['get'])
def get_polygon():
    conn= psycopg2.connect(host = '35.223.186.20',
                             database = 'lab0',
                             user = 'postgres',
                             password = '')

    cur = conn.cursor(cursor_factory=RealDictCursor)
    cur.execute(sql.SQL("SELECT mytable, ST_AsGeoJSON(geom)::json AS geometry FROM mytable WHERE id = %s"), (1,))
    result = cur.fetchone()
    cur.close()
    conn.close()
    return jsonify(result)

#name the .py file "app.py", then upload this app.py file to your ssh on google VM

In [ ]:
export FLASK_APP=app.py
flask run --host=0.0.0.0

In [ ]:
import requests

url = "http://34.27.164.176:5000/get_polygon"
response = requests.get(url)

if response.status_code == 200:
    geojson = response.json()
    with open("C:/Users/wang8837/Desktop/geokson/my_geojson_file.geojson", "w") as f:
        f.write(str(json))
        print("GeoJSON file saved to C:/my_geojson_file.geojson")
else:
    print("Failed to retrieve geojson data from the URL.")

In [ ]:
import json

# First JSON
json_string_1 = '{"geometry":{"coordinates":[[[[1.00012207,1.00012207],[2.00012207,1.00012207],[2.00012207,2.00012207],[1.00012207,2.00012207],[1.00012207,1.00012207]]]],"type":"MultiPolygon"},"mytable":"(1,0106000020E61000000100000001030000000100000005000000000000008000F03F000000008000F03F0000000040000040000000008000F03F00000000400000400000000040000040000000008000F03F0000000040000040000000008000F03F000000008000F03F)"}'

# Load the JSON string as a Python object
json_obj_1 = json.loads(json_string_1)

# Convert the first JSON object to the second JSON object
json_obj_2 = {
    "type": "FeatureCollection",
    "features": [
        {
            "type": "Feature",
            "geometry": json_obj_1['geometry'],
            "properties": {
                "mytable": json_obj_1['mytable']
            }
        }
    ]
}

# Convert the Python object to a JSON string
json_string_2 = json.dumps(json_obj_2)

# Print the resulting JSON string
print(json_string_2)

# Parse the JSON string into a dictionary
json_dict = json.loads(json_string_2)

# Save the dictionary as a JSON file
with open('C:/Users/wang8837/Desktop/geokson/esri_json.json', 'w') as json_file:
    json.dump(json_dict, json_file)
#This code will create a file called my_geojson_file.json in the current working directory and write the contents of json_dict to it in JSON format.






In [4]:
import arcpy
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection

# Connect to your ArcGIS Online organization
gis = arcgis.gis.GIS("home")

# Upload the GeoJSON file as a new item
item = gis.content.add({'type': 'GeoJson'}, 'C:/Users/Maochuan/OneDrive/桌面/7u.json')

# Publish the new item as a feature layer
feature_layer_item = item.publish()

# Get the feature layer URL and print it
feature_layer_url = feature_layer_item.url
print(feature_layer_url)

https://services.arcgis.com/8df8p0NlLFEShl0r/arcgis/rest/services/7u/FeatureServer
